# Quickstart/Cheat-Sheet

The gist of it

To start using `pyglotaran` in your project, you have to import it first.
In addition we need to import some extra components for later use.

In [ ]:
from glotaran.analysis.optimize import optimize
from glotaran.io import load_dataset
from glotaran.io import load_model
from glotaran.io import load_parameters
# To help with displaying yaml files
from glotaran.utils.ipython import display_file
# To help with plotting
from pyglotaran_extras import plot_data_overview
from pyglotaran_extras import plot_overview

The last import is a little ipython helper function to display files with syntax highlighting when running inside a jupyter-notebook.## Loading a dataset 

Let us get some example data to analyze:

## Generating a dataset 

We will generate a dataset by simulation used in the quickstart. This section will be hidden from the original quickstart, or included in the appendix at the end.

## Loading a dataset 

Let us get some example data to analyze:

Like all data in `pyglotaran`, the dataset is a [xarray.Dataset](https://xarray.pydata.org/en/stable/api.html#dataset).
You can find more information about the ``xarray`` library the [xarray hompage](https://xarray.pydata.org/en/stable/).

The loaded dataset is a simulated sequential model.

## Plotting raw data
Now we lets plot some time traces.

In [ ]:
plot_data = dataset.data.sel(spectral=[620, 630, 650], method="nearest")
plot_data.plot.line(x="time", aspect=2, size=5);

We can also plot spectra at different times.

In [ ]:
plot_data = dataset.data.sel(time=[1, 10, 20], method="nearest")
plot_data.plot.line(x="spectral", aspect=2, size=5);

## Preparing data
To get an idea about how to model your data, you should inspect the singular
value decomposition. Pyglotaran has a function to calculate it (among other
things).

In [ ]:
dataset = prepare_time_trace_dataset(dataset)
dataset

First, take a look at the first 10 singular values:

In [ ]:
plot_data = dataset.data_singular_values.sel(singular_value_index=range(0, 10))
plot_data.plot(yscale="log", marker="o", linewidth=0, aspect=2, size=5);

## Working with models

To analyze our data, we need to create a model. 

Create a file called `model.yaml` in your working directory and fill it with the following:

In [ ]:
display_file("model.yaml", syntax="yaml")

Now you can load the model file.

In [ ]:
model = load_model("model.yaml")

You can check your model for problems with `model.validate`.

In [ ]:
model.validate()

## Working with parameters

Now define some starting parameters. Create a file called `parameters.yaml` with
the following content.

In [ ]:
display_file("parameters.yaml", syntax="yaml")

In [ ]:
parameters = load_parameters("parameters.yaml")

You can `model.validate` also to check for missing parameters.

In [ ]:
model.validate(parameters=parameters)

Since not all problems in the model can be detected automatically it is wise to
visually inspect the model. For this purpose, you can just print the model.

In [ ]:
model

The same way you should inspect your parameters.

In [ ]:
parameters

## Optimizing data

Now we have everything together to optimize our parameters. First we import optimize.

In [ ]:
scheme = Scheme(model, parameters, {"dataset1": dataset})
result = optimize(scheme)
result

In [ ]:
result.optimized_parameters

You can get the resulting data for your dataset with `result.get_dataset`.

In [ ]:
result_dataset = result.data["dataset1"]
result_dataset

## Visualize the Result

The resulting data can be visualized the same way as the dataset. To judge the
quality of the fit, you should look at first left and right singular vectors of
the residual.

In [ ]:
residual_left = result_dataset.residual_left_singular_vectors.sel(left_singular_value_index=0)
residual_right = result_dataset.residual_right_singular_vectors.sel(right_singular_value_index=0)
residual_left.plot.line(x="time", aspect=2, size=5)
residual_right.plot.line(x="spectral", aspect=2, size=5);

Finally, you can save your result.

In [ ]:
save_dataset(result_dataset, "dataset1.nc")